### Homework 5

* Create a class and name it Z-test this class have 
* Create a class and name it T-test this class includes One-sample, two-sample, and paired t-test
* Create a class and name it ANOVA it includes one way and two ways
* Create a class and name it Chi-Square
* Create a class and name it AB testing 


In [1]:
from scipy import stats
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

class Ztest:
        
    def __init__(self, x, sigma, test_value = 0, alpha = 0.05, H1 = 'unequal', print_result = True):
        ''' 
        Runs a one-sample Z-test based on the specified input
        
        Input:
          x          : the data sample (a vector)
          sigma      : the standard deviation of the population from which x was drawn
          test_value : the hypothesized population mean
          alpha      : significance level for hypothesis testing
          H1         : alternative hypothesis: 'unequal' (default), 'smaller', or 'larger'
          print_result : if True, then results are printed to the console          
        '''
        # set attributes
        self.x = np.array(x)
        self.sigma = sigma
        self.test_value = test_value
        self.alpha = alpha
        self.H1 = H1
        # run the test 
        self.run_test()
        # show results
        if print_result:
            self.print_results()
        
    def run_test(self):
        '''Run the Z-test'''
        self.z = (self.x.mean() - self.test_value) / (self.sigma / np.sqrt(len(self.x)))        
        if self.H1 == 'unequal':
            self.p_value = 2 * stats.norm.sf(abs(self.z))
        elif self.H1 == 'smaller':
            self.p_value = stats.norm.cdf(self.z)
        elif self.H1 == 'larger':
            self.p_value = 1 - stats.norm.cdf(self.z)
        else:
            raise ValueError(f'Invalid alternative hypothesis: {self.H1}')        
        
    def print_results(self):
        '''Print the result of the statistical test'''
        print(f'\n---------------------------------')
        print(f'Z-test result')
        print(f'---------------------------------')
        print(f'H0: mu  =  {self.test_value}')
        if self.H1 == 'unequal':
            print(f'H1: mu =/= {self.test_value}')
        elif self.H1 == 'smaller':
            print(f'H1: mu < {self.test_value}')
        elif self.H1 == 'larger':
            print(f'H1: mu > {self.test_value}')
        else:
            raise ValueError(f'Invalid alternative hypothesis: {self.H1}')        
        print(f'sample mean = {self.x.mean():.2f} ± {np.std(self.x)/np.sqrt(len(self.x)):.2f} (n={len(self.x)})')
        print(f'sigma       = {self.sigma:.2f}')
        print(f'z           = {self.z:.2f}')
        print(f'p           = {self.p_value:.4f}')
        if self.p_value < self.alpha:
            print(f'When using alpha={self.alpha:.2f} as a rejection criterion, we would reject H0')
        else:
            print(f'When using alpha={self.alpha:.2f} as a rejection criterion, we would not reject H0') 


In [2]:
class Student_ttest:
        
    def __init__(self, x, alpha = 0.05, H1 = 'unequal', paired = False, test_value = 0, print_result = True):
        ''' 
        Runs a one- or two-sample Student t-test based on the specified input
        
        Input:
          x            : the data sample (a vector or a list with two vectors in case of a two-sample test)
          alpha        : significance level for hypothesis testing
          H1           : alternative hypothesis: 'unequal' (default), 'smaller', or 'larger'
          paired       : specifies whether to treat the samples as paired or as independent (only relevant if its a two-sample test)
          test_value   : the hypothesized population mean (only relevant if its a one-sample test
          print_result : if True, then results are printed to the console
        '''
        # set attributes
        self.x = x
        self.alpha = alpha
        self.H1 = H1
        self.is_paired = paired
        self.test_value = test_value
        self.is_two_sample_test = len(x) == 2
        # run a one-sample t-test on the difference scores if this is a paired-samples t-test
        if self.is_two_sample_test and self.is_paired:
            self.x = np.array(self.x[1]) - np.array(self.x[0])
            self.is_paired = False
            self.is_two_sample_test = False
        # run the test 
        self.run_test()
        # show results
        if print_result:
            self.print_results()
        
    def t_test_one_sample(self):
        '''Compute t-value and p-value for a one-sample t-test'''
        n = len(self.x)
        mean = np.mean(self.x)
        std = np.std(self.x, ddof=1)
        self.t_value = (mean - self.test_value) / (std / np.sqrt(n))
        df = n - 1
        if self.H1 == 'unequal':
            self.p_value = 2 * stats.t.sf(np.abs(self.t_value), df)
        elif self.H1 == 'smaller':
            self.p_value = 1 - stats.t.sf(np.abs(self.t_value), df)
        else:
            self.p_value = stats.t.sf(np.abs(self.t_value), df)

    def t_test_two_sample(self):
        '''Compute t-value and p-value for a two-sample t-test'''
        n1, n2 = len(self.x[0]), len(self.x[1])
        mean1, mean2 = np.mean(self.x[0]), np.mean(self.x[1])
        std1, std2 = np.std(self.x[0], ddof=1), np.std(self.x[1], ddof=1)
        pooled_std = np.sqrt(((n1 - 1) * std1 ** 2 + (n2 - 1) * std2 ** 2) / (n1 + n2 - 2))
        self.t_value = (mean1 - mean2) / (pooled_std * np.sqrt(1 / n1 + 1 / n2))
        df = n1 + n2 - 2
        if self.H1 == 'unequal':
            self.p_value = 2 * stats.t.sf(np.abs(self.t_value), df)
        elif H1 == 'smaller':
            self.p_value = 1 - stats.t.sf(np.abs(self.t_value), df)
        else:
            self.p_value = stats.t.sf(np.abs(self.t_value), df)

    def run_test(self):
        '''Run the t-test'''
        if not self.is_two_sample_test:
            self.t_test_one_sample()
        else:
            self.t_test_two_sample()

    def print_results(self):
        '''Print the result of the statistical test'''
        print(f'\n---------------------------------')
        print(f'Student t-test result')
        print(f'---------------------------------')
        if self.is_two_sample_test:
            print(f'H0: mu1 = mu2')
        else:
            print(f'H0 = {self.test_value}')
        print(f'H1: {self.H1}')
        if self.is_two_sample_test:
            print(f'sample mean1 = {np.mean(self.x[0]):.2f} ± {np.std(self.x[0])/np.sqrt(len(self.x[0])):.2f} (n={len(self.x[0])})')
            print(f'sample mean2 = {np.mean(self.x[1]):.2f} ± {np.std(self.x[1])/np.sqrt(len(self.x[1])):.2f} (n={len(self.x[1])})')
        else:
            print(f'sample mean  = {np.mean(self.x):.2f} ± {np.std(self.x)/np.sqrt(len(self.x)):.2f} (n={len(self.x)})')
        print(f't            = {self.t_value:.2f}')
        print(f'p            = {self.p_value:.4f}')
        if self.p_value < self.alpha:
            if self.is_two_sample_test:
                print(f'When using alpha={self.alpha:.2f} as a rejection criterion, we would reject the null hypothesis that mu1 = mu2')
            else:
                print(f'When using alpha={self.alpha:.2f} as a rejection criterion, we would reject the null hypothesis that mu = {self.test_value}')
        else:
            if self.is_two_sample_test:
                print(f'When using alpha={self.alpha:.2f} as a rejection criterion, we would not reject the null hypothesis that mu1 = mu2')
            else:
                print(f'When using alpha={self.alpha:.2f} as a rejection criterion, we would not reject the null hypothesis that mu = {self.test_value}')


In [3]:
class ANOVA:
        
    def __init__(self, x, groups, alpha = 0.05, factor_names = [], print_result = True):
        ''' 
        Runs a one- or multiway ANOVA based on the specified input
        
        Input:
          x            : the data sample (a vector or a list with two vectors in case of a two-sample test)
          groups       : a list of lists, each with the same length as x; each list is a dependent variable and each element specifies the level for the i-th datapoint
          alpha        : significance level for hypothesis testing
          print_result : if True, then results are printed to the console
        '''
        # set attributes
        self.x = x
        self.groups = groups
        self.alpha = alpha
        self.factor_names = factor_names
        # make sure that groups is a list of lists, even if it's a one-way ANOVA
        if not isinstance(self.groups[0],list):
            self.groups = [self.groups]
        # make sure that factor_names is a list of strings even if no factor names were specified
        if self.factor_names == []:
            self.factor_names = ['factor' + str(ii+1) for ii in range(0,len(self.groups))]
        # run the test 
        self.run_test()
        # show results
        if print_result:
            self.print_results()
        
    def oneway(self):
        '''Perform a one-way ANOVA (without external packages)'''
        k = len(np.unique(self.groups[0]))
        df_between = k - 1
        df_within = len(self.x) - k
        grand_mean = np.mean(self.x)
        # compute sum-of-squares between subjects
        SS_between = 0
        for group in self.groups:
            group_mean = np.mean([self.x[ii] for ii, g in enumerate(group) if g == group[0]])
            SS_between += len([g for g in group if g == group[0]]) * (group_mean - grand_mean) ** 2
        # compute sum-of-squares within subjects
        SS_within = 0
        for group in self.groups:
            group_mean = np.mean([self.x[ii] for ii, g in enumerate(group) if g == group[0]])
            SS_within += sum([(self.x[i] - group_mean) ** 2 for i, g in enumerate(group) if g == group[0]])        
        # compute F value as the ratio between the mean between and mean within variance
        MS_between = SS_between/df_between
        MS_within = SS_within/df_within
        self.F_value = MS_between/MS_within
        # compute p value
        self.p_value = 1 - stats.f.cdf(self.F_value, df_between, df_within)      

    def twoway(self):
        '''Perform a two-way ANOVA (using ols)'''
        data = pd.DataFrame({'response': self.x, self.factor_names[0]: self.groups[0], self.factor_names[1]: self.groups[1]})
        formula = f'response ~ C({self.factor_names[0]}) + C({self.factor_names[1]}) + C({self.factor_names[0]}):C({self.factor_names[1]})'
        model = smf.ols(formula, data).fit()        
        anova_table = sm.stats.anova_lm(model, typ=2)
        self.F_value = anova_table['F'].tolist()
        self.p_value = anova_table['PR(>F)'].tolist()

    def run_test(self):
        '''Run the t-test'''
        if len(self.groups) == 1:
            self.oneway()
        else:
            self.twoway()

    def print_results(self):
        '''Print the result of the statistical test'''
        print('\n---------------------------------')
        print('ANOVA test result')
        print('---------------------------------')
        print(f' {"Factor":15}  {"F-value":>10}  {"p-value":>10}')
        if isinstance(self.F_value, list):
            for i, factor in enumerate(self.factor_names):
                print(f' {factor:15}  {self.F_value[i]:10.4f}  {self.p_value[i]:10.4f}')
            if len(self.groups) == 2:
                interaction_factor = f'{self.factor_names[0]}*{self.factor_names[1]}'
                print(f' {interaction_factor:15}  {self.F_value[-2]:10.4f}  {self.p_value[-2]:10.4f}')                
        else:
            factor = self.factor_names[0]
            print(f' {factor:15}  {self.F_value:10.4f}  {self.p_value:10.4f}')    

In [4]:
class ChiSquare:
        
    def __init__(self, x, expected_values, alpha = 0.05, print_result = True):
        ''' 
        Runs a chi-square test to test the null hypothesis that the given sample was drawn from the specified distribution
        
        Input:
          x               : observed counts
          expected_counts : expected counts
          alpha           : significance level for hypothesis testing
          print_result    : if True, then results are printed to the console
        '''
        # set attributes
        self.observed_counts = x
        self.expected_counts = expected_values
        self.alpha = alpha
        # run the test 
        self.run_test()
        # show results
        if print_result:
            self.print_results()
        
    def run_test(self):
        '''Run the chi-square test'''
        chi_square_statistic = 0
        for i in range(len(self.observed_counts)):
            observed = self.observed_counts[i]
            expected = self.expected_counts[i]
            chi_square_statistic += (observed - expected)**2 / expected
        self.chi_square_statistic = chi_square_statistic
        self.p_value = 1 - stats.chi2.cdf(self.chi_square_statistic, len(self.observed_counts) - 1)
      
    def print_results(self):
        '''Print the result of the statistical test'''
        print('\n---------------------------------')
        print('Chi-square test result')
        print('---------------------------------')
        print('H0: The sample was drawn from the specified distribution')
        print(f' {"Test statistic":20}  {self.chi_square_statistic:.4f}')
        print(f' {"p-value":20}  {self.p_value:.4f}')
        if self.p_value < self.alpha:
            print(f'When using alpha={self.alpha:.2f} as a rejection criterion, we would reject H0')
        else:
            print(f'When using alpha={self.alpha:.2f} as a rejection criterion, we would not reject H0')            


In [5]:
# Testing out the classes on the assignment data (code from here on is not final)

x = [4.186, 4.439, 4.781, 4.388, 4.947, 4.853, 4.889, 4.682, 4.428,  4.533, 4.557, 4.761, 4.491, 4.334, 4.83 , 4.268, 4.68 , 4.437, 5.382, 5.111, 5.096, 5.232, 5.033, 5.57 , 4.474, 4.789, 4.725, 4.84 , 4.817, 4.438, 4.754, 4.966, 4.285, 4.482, 4.396, 4.418, 4.514, 5.383, 5.264, 4.309, 5.058, 4.392, 4.788, 4.934, 4.967, 4.554, 4.42 , 5. , 5.126, 5.082, 4.944, 4.658]
ztest_result = Ztest(x = x, sigma = .6, test_value = 4.5, alpha = .05, H1 = 'unequal')

# independent t-test
new_flavor =[8, 7, 9, 6, 7, 8, 9, 7, 8, 7, 6, 8, 7, 9, 8, 7, 6, 9, 8, 7]
best_selling_flavor = [6, 7, 8, 6, 7, 6, 7, 6, 8, 7, 6, 7, 6, 8, 7, 6, 7, 8, 6, 7]
ttest = Student_ttest(x = [new_flavor, best_selling_flavor], alpha = 0.05, H1 = 'unequal', paired = False)

# paired t-test
before = [15, 18, 12, 10, 17, 16, 12, 14, 19, 18, 11, 13, 16, 17, 19, 14, 16, 13, 15, 12]
after = [18, 20, 15, 13, 19, 18, 14, 16, 21, 20, 14, 16, 19, 20, 22, 16, 18, 15, 17, 14]
ttest = Student_ttest(x = [before, after], alpha = 0.05, H1 = 'unequal', paired = True)

# one-way ANOVA
dept_a = [55, 60, 50, 58, 63, 62, 57, 56, 61, 59]
dept_b = [50, 52, 48, 49, 55, 53, 51, 54, 47, 50]
dept_c = [45, 43, 48, 50, 42, 47, 49, 46, 44, 48]
groups = ['A']*len(dept_a) + ['B']*len(dept_b) + ['C']*len(dept_c)
x = dept_a + dept_b + dept_c
anova = ANOVA(x, groups, 'department')

# two-way ANOVA
dept_a_m = [55, 60, 50, 58, 63]
dept_a_f = [62, 57, 56, 61, 59]
dept_b_m = [50, 52, 48, 49, 55]
dept_b_f = [53, 51, 54, 47, 50]
dept_c_m = [45, 43, 48, 50, 42]
dept_c_f = [47, 49, 46, 44, 48]
x = dept_a_m + dept_a_f + dept_b_m + dept_b_f + dept_c_m + dept_c_f
factor1 = [1]*len(dept_a_m+dept_a_f) + [2]*len(dept_b_m+dept_b_f) + [3]*len(dept_c_m+dept_c_f)
factor2 = [1]*len(dept_a_m) + [2]*len(dept_a_f) + [1]*len(dept_b_m) + [2]*len(dept_b_f) + [1]*len(dept_c_m) + [2]*len(dept_c_f)
anova = ANOVA(x, [factor1, factor2], factor_names = ['dept', 'gender'])

# Chi-square
x = [18, 20, 16, 22, 14, 10]
expected = [100/6]*6
chisquare = ChiSquare(x,expected)

# A/B testing (i was lazy and did this without implementing a separate class for it)
old_flavor = [6, 7, 8, 5, 6, 7, 5, 8, 6, 7, 5, 6, 7, 6, 5]
new_flavor = [8, 9, 7, 8, 9, 6, 7, 8, 7, 8, 7, 8, 9, 6, 8]
ttest = Student_ttest(x = [new_flavor, old_flavor], alpha = 0.05, H1 = 'unequal', paired = False)


---------------------------------
Z-test result
---------------------------------
H0: mu  =  4.5
H1: mu =/= 4.5
sample mean = 4.74 ± 0.05 (n=52)
sigma       = 0.60
z           = 2.93
p           = 0.0034
When using alpha=0.05 as a rejection criterion, we would reject H0

---------------------------------
Student t-test result
---------------------------------
H0: mu1 = mu2
H1: unequal
sample mean1 = 7.55 ± 0.22 (n=20)
sample mean2 = 6.80 ± 0.17 (n=20)
t            = 2.66
p            = 0.0113
When using alpha=0.05 as a rejection criterion, we would reject the null hypothesis that mean1 = mean2

---------------------------------
Student t-test result
---------------------------------
H0 = 0
H1: unequal
sample mean  = 2.40 ± 0.11 (n=20)
t            = 21.35
p            = 0.0000
When using alpha=0.05 as a rejection criterion, we would reject the null hypothesis that mean = 0

---------------------------------
ANOVA test result
---------------------------------
 Factor              F-val